In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import sys
import math
import numpy as np

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset_circles.zip to dataset_circles.zip


In [4]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['dataset_circles.zip']), 'r')
data.extractall()

In [5]:
dir_dataset_circles_directory = "/content/dataset_circles"

In [30]:
def detectCircles(image_name, directory):
  dir_image = f'{directory}/{image_name}'  
  # Read image as gray-scale
  img = cv2.imread(dir_image, cv2.IMREAD_COLOR)  
  # Convert to gray-scale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  #plt.imshow(gray)
  #plt.show()
  # Blur the image to reduce noise
  img_blur = cv2.medianBlur(gray, 1)
  #plt.imshow(img_blur)
  #plt.show()   
  # Apply hough transform on the image
  circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, img.shape[0]/64, param1=200, param2=10, minRadius=5, maxRadius=30)
  circlesToReturn = 0
  # Draw detected circles
  if circles is not None:
      #print(circles.size)
      circlesToReturn = circles.size
      circles = np.uint16(np.around(circles))
      for i in circles[0, :]:
          # Draw outer circle
          cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
          # Draw inner circle
          cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 3)
  else:
     print("No circles")
  
  return circlesToReturn

  #plt.imshow(img)
  #plt.show() 

In [35]:
circlesModel = detectCircles("circulo.png", "/content/")
toPrintModel = f'Model---{circlesModel}' 
print(toPrintModel)


Model---138


In [40]:
dir_circles_images = os.listdir(dir_dataset_circles_directory)
for image_name in dir_circles_images:
  if image_name.startswith("nosano"):
    circles_nosanos = detectCircles(image_name, dir_dataset_circles_directory)
    percent = round(circles_nosanos / circlesModel, 2)
    toPrint = f'{image_name}---{circles_nosanos} -- {percent}' 
    print(toPrint)
    

nosanos-id39-circulo3.png---99 -- 0.72
nosanos-id235-circulo3.png---225 -- 1.63
nosanos-id235-circulo1.png---318 -- 2.3
nosanos-id416-circulo1.png---81 -- 0.59
nosanos-id105-circulo1.png---63 -- 0.46
nosanos-id34-circulo4.png---99 -- 0.72
nosanos-id416-circulo3.png---327 -- 2.37
nosanos-id139-circulo1.png---51 -- 0.37
nosanos-id109-circulo2.png---63 -- 0.46
nosanos-id229-circulo2.png---57 -- 0.41
nosanos-id39-circulo5.png---81 -- 0.59
nosanos-id39-circulo2.png---132 -- 0.96
nosanos-id214-circulo4.png---81 -- 0.59
nosanos-id139-circulo2.png---51 -- 0.37
nosanos-id119-circulo1.png---81 -- 0.59
nosanos-id27-circulo3.png---48 -- 0.35
nosanos-id34-circulo3.png---66 -- 0.48
nosanos-id15-circulo3.png---150 -- 1.09
nosanos-id139-circulo3.png---69 -- 0.5
nosanos-id117-circulo5.png---93 -- 0.67
nosanos-id119-circulo6.png---66 -- 0.48
nosanos-id39-circulo4.png---78 -- 0.57
nosanos-id414-circulo2.png---87 -- 0.63
nosanos-id425-circulo3.png---186 -- 1.35
nosanos-id527-circulo3.png---78 -- 0.57
nosa

In [42]:
for image_name in dir_circles_images:
  if image_name.startswith("sano"):    
    circles_sanos = detectCircles(image_name, dir_dataset_circles_directory)
    percent_sanos = round(circles_sanos / circlesModel, 2)
    toPrint = f'{image_name}---{circles_sanos} -- {percent_sanos}' 
    print(toPrint)

sanos-id2-circulo4.png---15 -- 0.11
sanos-id113-circulo5.png---42 -- 0.3
sanos-id3-circulo2.png---87 -- 0.63
sanos-id107-circulo1.png---81 -- 0.59
sanos-id122-circulo4.png---57 -- 0.41
sanos-id31-circulo5.png---78 -- 0.57
sanos-id101-circulo4.png---51 -- 0.37
sanos-id33-circulo4.png---78 -- 0.57
sanos-id101-circulo2.png---54 -- 0.39
sanos-id108-circulo3.png---66 -- 0.48
sanos-id110-circulo1.png---60 -- 0.43
sanos-id102-circulo1.png---57 -- 0.41
sanos-id118-circulo2.png---60 -- 0.43
sanos-id110-circulo3.png---63 -- 0.46
sanos-id108-circulo1.png---42 -- 0.3
sanos-id29-circulo1.png---54 -- 0.39
sanos-id33-circulo3.png---48 -- 0.35
sanos-id103-circulo4.png---69 -- 0.5
sanos-id28-circulo4.png---33 -- 0.24
sanos-id102-circulo3.png---72 -- 0.52
sanos-id121-circulo3.png---57 -- 0.41
sanos-id29-circulo3.png---81 -- 0.59
sanos-id122-circulo5.png---51 -- 0.37
sanos-id122-circulo1.png---66 -- 0.48
sanos-id28-circulo2.png---39 -- 0.28
sanos-id118-circulo1.png---72 -- 0.52
sanos-id28-circulo3.png---